In [4]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

Requirement already up-to-date: tables in /usr/local/lib/python3.6/dist-packages (3.6.1)


In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae 
from sklearn.model_selection import cross_val_score, KFold

import eli5 
from eli5.sklearn import PermutationImportance



In [10]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [12]:
cd"/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car


In [0]:
df = pd.read_hdf('data/car.h5')

In [14]:
df.shape

(106494, 155)

Feature Engineering

In [0]:
SUFFIX_CAT = '__cat'

for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values 

In [17]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats) 

151

In [22]:
X = df[cat_feats].values
y = df['price_value'].values

model = DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
np.mean(scores), np.std(scores)

(-19747.81093847179, 75.95212671816309)

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values
  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

Decision Tree

In [25]:
run_model( DecisionTreeRegressor(max_depth=5), cat_feats)

(-19747.81093847179, 75.95212671816309)

Random Forest

In [26]:
model = RandomForestRegressor(max_depth=5, n_estimators=50, random_state=0)
run_model(model, cat_feats)

(-18864.27843495869, 38.24233578175403)

XGBOOST

In [29]:
xgb_params = {
    'max_depth':5,
    'n_estimaros': 50,
    'learning_rate': 0.1,
    'seed': 0 
}

run_model(xgb.XGBRegressor(**xgb_params), cat_feats)

[19:17:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:18:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:18:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11679.963838633514, 102.88480462583918)

In [31]:
m = xgb.XGBRegressor(max_depth=5, n_estimators=50, learning_rate=0.1, seed=0)
m.fit(X, y)

imp = PermutationImportance(m, random_state=0).fit(X, y)
eli5.show_weights(imp, feature_names=cat_feats)

[19:20:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1274 ± 0.0017,param_napęd__cat
0.1159 ± 0.0025,param_stan__cat
0.1131 ± 0.0037,param_rok-produkcji__cat
0.0565 ± 0.0010,param_faktura-vat__cat
0.0482 ± 0.0014,param_moc__cat
0.0416 ± 0.0017,param_skrzynia-biegów__cat
0.0252 ± 0.0010,param_marka-pojazdu__cat
0.0247 ± 0.0014,feature_kamera-cofania__cat
0.0232 ± 0.0008,param_typ__cat
0.0198 ± 0.0006,param_pojemność-skokowa__cat


In [33]:
feats = ['param_napęd__cat', 'param_stan__cat', 'param_rok-produkcji__cat', 'param_faktura-vat__cat', 'param_moc__cat', 'param_skrzynia-biegów__cat', 'param_marka-pojazdu__cat', 'feature_kamera-cofania__cat', 'param_typ__cat', 'param_pojemność-skokowa__cat', 'seller_name__cat', 'param_wersja__cat', 'feature_wspomaganie-kierownicy__cat', 'param_model-pojazdu__cat', 'feature_system-start-stop__cat', 'param_kod-silnika__cat', 'feature_asystent-pasa-ruchu__cat', 'feature_łopatki-zmiany-biegów__cat', 'feature_światła-led__cat', 'feature_czujniki-parkowania-przednie__cat']
run_model(xgb.XGBRegressor(**xgb_params), feats)

[19:40:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:40:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:40:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11968.688449195286, 125.34058902441106)

In [35]:
df['param_napęd'].unique()

array([None, 'Na przednie koła', '4x4 (dołączany automatycznie)',
       'Na tylne koła', '4x4 (dołączany ręcznie)', '4x4 (stały)'],
      dtype=object)

In [37]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x)== 'None' else int())
df.shape

(106494, 309)

In [38]:
feats = ['param_napęd__cat', 'param_stan__cat', 'param_rok-produkcji', 'param_faktura-vat__cat', 'param_moc__cat', 'param_skrzynia-biegów__cat', 'param_marka-pojazdu__cat', 'feature_kamera-cofania__cat', 'param_typ__cat', 'param_pojemność-skokowa__cat', 'seller_name__cat', 'param_wersja__cat', 'feature_wspomaganie-kierownicy__cat', 'param_model-pojazdu__cat', 'feature_system-start-stop__cat', 'param_kod-silnika__cat', 'feature_asystent-pasa-ruchu__cat', 'feature_łopatki-zmiany-biegów__cat', 'feature_światła-led__cat', 'feature_czujniki-parkowania-przednie__cat']
run_model(xgb.XGBRegressor(**xgb_params), feats)

[19:45:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:45:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:45:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-14546.458784844124, 93.70410570055135)

In [0]:
df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]))

In [47]:
#df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]))

df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x)== 'None' else int())

feats = ['param_napęd__cat', 'param_stan__cat', 'param_rok-produkcji', 'param_faktura-vat__cat', 'param_moc', 'param_skrzynia-biegów__cat', 'param_marka-pojazdu__cat', 'feature_kamera-cofania__cat', 'param_typ__cat', 'param_pojemność-skokowa__cat', 'seller_name__cat', 'param_wersja__cat', 'feature_wspomaganie-kierownicy__cat', 'param_model-pojazdu__cat', 'feature_system-start-stop__cat', 'param_kod-silnika__cat', 'feature_asystent-pasa-ruchu__cat', 'feature_łopatki-zmiany-biegów__cat', 'feature_światła-led__cat', 'feature_czujniki-parkowania-przednie__cat']
run_model(xgb.XGBRegressor(**xgb_params), feats)

[19:55:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:55:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:56:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13423.005865932404, 37.00118062056733)

In [49]:
df['param_pojemność-skokowa'].unique()

array(['898 cm3', '1 560 cm3', '3 000 cm3', ..., '5 992 cm3', '1 966 cm3',
       '142 280 cm3'], dtype=object)

In [0]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(x.split('cm')[0].replace(' ', '')))

In [52]:
feats = ['param_napęd__cat', 'param_stan__cat', 'param_rok-produkcji', 'param_faktura-vat__cat', 'param_moc', 'param_skrzynia-biegów__cat', 'param_marka-pojazdu__cat', 'feature_kamera-cofania__cat', 'param_typ__cat', 'param_pojemność-skokowa', 'seller_name__cat', 'param_wersja__cat', 'feature_wspomaganie-kierownicy__cat', 'param_model-pojazdu__cat', 'feature_system-start-stop__cat', 'param_kod-silnika__cat', 'feature_asystent-pasa-ruchu__cat', 'feature_łopatki-zmiany-biegów__cat', 'feature_światła-led__cat', 'feature_czujniki-parkowania-przednie__cat']
run_model(xgb.XGBRegressor(**xgb_params), feats)

[20:01:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:01:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:01:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13091.529294030202, 44.50712532931837)